In [3]:
import math
import heapq
import numpy as np
from loguru import logger
from unidecode import unidecode
import re
import random

def setup_logger(file_name):
    logger.remove()
    logger.add(f"./logs/{file_name}-debug.log", format="{time} {level} {message}", level="DEBUG", enqueue=True)
    logger.add(f"./logs/{file_name}-info.log", format="{time} {level} {message}", level="INFO", enqueue=True, backtrace=True)
    logger.add(f"./logs/{file_name}-error.log", format="{time} {level} {message}", level="ERROR", enqueue=True, backtrace=True, diagnose=True)
    
setup_logger("hw1")

def get_ind(ndarray, value):
    return list(zip(*np.where(ndarray == value)))

def p2i(perm):
    k = 0
    m = 1
    n = len(perm)
    pos = [0]*n
    elems = [0]*n

    for i in range(n):
        pos[i] = i
        elems[i] = i

    for i in range(n - 1):
            k += m*pos[perm[i] - 1]
            m = m*(n-i)
            pos[elems[n-i-1]] = pos[perm[i] - 1]
            elems[pos[perm[i] - 1]] = elems[n-i-1]
    return k
    
class State:
    def __init__(self, nums, n):
        self.nums = nums
        self.n = n
        self.id = p2i(nums.flatten())
    
    def get_childs(self, c):
        i, j = get_ind(self.nums, n*n)[0]
        def make(nums, n, p, v):
            action = ''
            if v == (-1, 0):
                action = 'U'
            if v == (1, 0):
                action = 'D'
            if v == (0, -1):
                action = 'L'
            if v == (0, 1):
                action = 'R'
            tmp = nums[p[0], p[1]].copy()
            nums[p[0], p[1]] = nums[p[0] + v[0], p[1] + v[1]].copy()
            nums[p[0] + v[0], p[1] + v[1]] = tmp
            if nums[p[0], p[1]] == c or nums[p[0] + v[0], p[1] + v[1]] == c:
                action += " C"
            return State(nums, self.n), action 
        childs = []
        if i < n - 1:
            childs.append(make(self.nums.copy(), self.n, (i, j), (1, 0)))
        if i > 0:
            childs.append(make(self.nums.copy(), self.n, (i, j), (-1, 0)))
        if j < n - 1:
            childs.append(make(self.nums.copy(), self.n, (i, j), (0, 1)))
        if j > 0:
            childs.append(make(self.nums.copy(), self.n, (i, j), (0, -1)))
        return childs
    
    def __eq__(self, other):
        if isinstance(other, State):
            return self.id == other.id
        return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    def __str__(self):
        return str(self.nums)

class Frontier:
    def __init__(self, alg_type):
        self.q = []
        self.alg_type = alg_type
    
    def add(self, node):
        if self.alg_type == "A*":
            heapq.heappush(self.q, (node.total_cost, node))
        elif self.alg_type == "UCS":
            heapq.heappush(self.q, (node.cost, node))
        else:
            raise
    
    def pop(self):
        return heapq.heappop(self.q)[1]
    
    def is_empty(self):
        return len(self.q) == 0

class Node:
    
    def __init__(self, state, cost, parent, c, action=''):
        def calc_huristic(state):
            h = 0
            for i in range(state.n):
                for j in range(state.n):
                    if state.nums[i, j] != i*state.n + j + 1:
                        h += 1
            return h
        
        self.state = state
        self.cost = cost
        self.huristic = calc_huristic(state)
        self.total_cost = cost + self.huristic
        self.parent = parent
        self.c = c
        self.action = action
    
    def expand(self):
        nodes = []
        childs_state = self.state.get_childs(self.c)
        for state, action in childs_state:
            nodes.append(Node(state, self.cost + 1, self, self.c, action=action))
        return nodes
    
    def __lt__(self, other):
        return str((self.cost<other.cost))
    
    def __str__(self):
        return str(self.state.nums) + " " + str(self.cost) + " " + str(self.action)
    
    def __eq__(self, other):
        if isinstance(other, Node):
            return self.state == other.state
        return False
    
    def __ne__(self, other):
        return not self.__eq__(other)


    
def run(start_node, alg_type):
    n = start_node.state.n
    PROBLEM_ANSWER = p2i(range(1, n*n+1))
    
    def is_goal(node):
        n = node.state.n
        return node.state.id == PROBLEM_ANSWER
    
    dones = {}
    f = Frontier(alg_type)
    f.add(start_node)
    while not f.is_empty():
        if len(dones) > 10000:
            return False
        if len(dones) % 100 == 0:
            logger.debug(str(len(dones)) + "-" + str(len(f.q)))
        node = f.pop()
        if is_goal(node):
            return node, len(dones)
        childs = node.expand()
        for child in childs:
            if not dones.get(child.state.id, False) and child.state.id != node.state.id:
                f.add(child)
        dones[node.state.id] = True

def ucs(start_node):
    return run(start_node, "UCS")
    
def Astar(start_node):
    return run(start_node, "A*")
        
# nums_list = list(range(1, n*n+1))[::-1]
# nums = np.array([[4, 2], [1, 3]])
# nums = np.array([[8, 1, 2], [9, 4, 3], [7, 6, 5]])
# nums = np.array([[1, 8, 2], [9, 4, 3], [7, 6, 5]])
# nums = [1, 8, 2, 9, 4, 3, 6, 7, 5]
retry = 10
while retry > 0:
    n = 3
    nums = list(range(1, n*n+1))
    random.shuffle(nums)
    c = 3
    # n = int(input())
    # nums = [int(x) for x in input().split(" ")]
    nums = np.array([nums[i*n:i*n+n] for i in range(n)])
    # c = int(input())
    s_node = Node(State(nums, n), 0, None, c)
    ans = Astar(s_node)
    if not ans:
        pass
#         print("Unsolvable")
    else:
        ans, len_dones = ans
        retry -= 1
        actions = ""
        x = ans
        while x.parent != None:
            actions = x.action + " " + actions
            x = x.parent
        print(n, nums.flatten(), c)
        print(actions)
        print(len_dones)

3 [5 1 9 4 8 3 6 2 7] 3
D C L D L U U R D D R U L L D R R 
366
3 [1 5 8 3 7 6 2 4 9] 3
U U L L D C D R U U L C D R R U L C L D D R U R D 
6987
3 [8 3 1 4 5 2 7 9 6] 3
U U C R D L C D L U U R R D C L L D R U R D 
2128
3 [9 1 5 2 7 4 8 6 3] 3
D R U R D D C L L U R U R D C L U L D R R D 
3217
3 [3 1 2 5 8 7 4 9 6] 3
U L U C R D R D L U L C D R R U L C U R D C D 
1928
3 [2 3 4 8 7 5 9 6 1] 3
U R U C R D D L L U U R D C R U L C D L U R D R D 
6863
3 [8 7 3 6 9 4 2 1 5] 3
D L U U R D D L U U R D R D L L U R R D 
2961
3 [2 5 1 3 8 9 6 4 7] 3
L L C D R R U L C L U R R D C L L U R D L D R R 
4173
3 [8 1 4 7 2 5 3 9 6] 3
L C U U R D D C R U L C L U R R D C D L U U L D R R D 
8981
3 [7 1 5 4 3 9 6 8 2] 3
D L U C L U R R D D L C L U R R D C L U U R D C D 
4288
